In [5]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

raw_csv_data = np.loadtxt('e-books-data.csv', delimiter=',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]
print(unscaled_inputs_all.shape)
print(targets_all.shape)

(14084, 10)
(14084,)


In [6]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter =0
indices_to_remove =[]

for i in range(targets_all.shape[0]):

    if targets_all[i]==0:        
        zero_targets_counter +=1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i) # Deleting these indices will balance the dataset
        

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all,indices_to_remove, axis =0)   
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis =0)

In [7]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [8]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [27]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count =int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count 

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
# They are the next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Create variables that record the inputs and targets for test.
# They are everything that is remaining.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]


In [28]:
# We have saved our dataset so we can use it
np.savez('e-books_data_train', inputs=train_inputs, targets=train_targets)
np.savez('e-books_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('e-books_data_test', inputs=test_inputs, targets=test_targets)

In [29]:
train_inputs = train_inputs.astype(float)
train_targets = train_targets.astype(float)

validation_inputs = validation_inputs.astype(float)
validation_targets = validation_targets.astype(float)

test_inputs = test_inputs.astype(float)
test_targets = test_targets.astype(float)

### Model 

In [31]:
# Set the input and output sizes
input_size = 10
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


### Choose the optimizer and the loss function

# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### Training Model

In [37]:
### Training
# That's where we train the model we have built.

# set the batch size
batch_size = 100

# set a maximum number of training epochs
max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=3) #It will stop the model if val_loss is getting bigger 

# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size          
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          callbacks =[early_stopping],
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )  

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 0s - loss: 0.2020 - accuracy: 0.9243 - val_loss: 0.2270 - val_accuracy: 0.9150
Epoch 2/100
3579/3579 - 0s - loss: 0.1997 - accuracy: 0.9248 - val_loss: 0.2399 - val_accuracy: 0.9105
Epoch 3/100
3579/3579 - 0s - loss: 0.2001 - accuracy: 0.9262 - val_loss: 0.2295 - val_accuracy: 0.9128
Epoch 4/100
3579/3579 - 0s - loss: 0.2023 - accuracy: 0.9232 - val_loss: 0.2259 - val_accuracy: 0.9150
Epoch 5/100
3579/3579 - 0s - loss: 0.2050 - accuracy: 0.9204 - val_loss: 0.2343 - val_accuracy: 0.9172
Epoch 6/100
3579/3579 - 0s - loss: 0.1990 - accuracy: 0.9248 - val_loss: 0.2272 - val_accuracy: 0.9128
Epoch 7/100
3579/3579 - 0s - loss: 0.2007 - accuracy: 0.9257 - val_loss: 0.2385 - val_accuracy: 0.9128


### Testing Model

In [38]:
test_loss, test_accuracy = model.evaluate(test_inputs,test_targets)

448/448 [==============================] - 0s 56us/sample - loss: 0.2573 - accuracy: 0.8996
